In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

root_path = 'drive/MyDrive/NBA-predictions'

Mounted at /content/drive


In [3]:
df = pd.read_csv(root_path +'/data/2013_ppdf.csv')

In [4]:
df.head()

,GAME_DATE,GAME_ID,MATCHUP,HOME_TEAM,AWAY_TEAM,HOME_PREV_SEAS_WIN_PERC,HOME_PREV_SEAS_AVG_PTS_PER_GAME,HOME_PREV_SEAS_FG_PERC,HOME_PREV_SEAS_THREE_PERC,HOME_PREV_SEAS_FREE_THROW_PERC,HOME_PREV_SEAS_PLUS_MINUS,AWAY_PREV_SEAS_WIN_PERC,AWAY_PREV_SEAS_AVG_PTS_PER_GAME,AWAY_PREV_SEAS_FG_PERC,AWAY_PREV_SEAS_THREE_PERC,AWAY_PREV_SEAS_FREE_THROW_PERC,AWAY_PREV_SEAS_PLUS_MINUS,HOME_WIN
0,10/29/2013,21300001,ORL @ IND,IND,ORL,0.605,94.7,43.6,34.7,74.6,4.0,0.244,94.1,44.8,32.9,75.5,-7.0,1
1,10/29/2013,21300003,LAC @ LAL,LAL,LAC,0.549,102.2,45.8,35.5,69.2,1.2,0.683,101.1,47.8,35.8,71.1,6.5,1
2,10/29/2013,21300002,CHI @ MIA,MIA,CHI,0.805,102.9,49.6,39.6,75.4,7.9,0.549,93.2,43.7,35.3,77.3,0.3,1
3,10/30/2013,21300015,POR @ PHX,PHX,POR,0.305,95.2,44.3,33.0,74.4,-6.5,0.402,97.5,44.8,35.3,77.6,-3.2,1
4,10/30/2013,21300016,DEN @ SAC,SAC,DEN,0.341,100.2,44.7,36.3,76.9,-4.9,0.695,106.1,47.8,34.3,70.1,5.1,1


In [5]:
AWAY_TEAM_DICT = {
    'ATL' : 'AWAY_ATL',
    'BKN' : 'AWAY_BKN',
    'BOS' : 'AWAY_BOS', 
    'CHA' : 'AWAY_CHA', 
    'CHI' : 'AWAY_CHI', 
    'CLE' : 'AWAY_CLE', 
    'DAL' : 'AWAY_DAL', 
    'DEN' : 'AWAY_DEN', 
    'DET' : 'AWAY_DET', 
    'GSW' : 'AWAY_GSW', 
    'HOU' : 'AWAY_HOU', 
    'IND' : 'AWAY_IND', 
    'LAC' : 'AWAY_LAC', 
    'LAL' : 'AWAY_LAL', 
    'MEM' : 'AWAY_MEM', 
    'MIA' : 'AWAY_MIA', 
    'MIL' : 'AWAY_MIL', 
    'MIN' : 'AWAY_MIN', 
    'NOP' : 'AWAY_NOP', 
    'NYK' : 'AWAY_NYK', 
    'OKC' : 'AWAY_OKC', 
    'ORL' : 'AWAY_ORL', 
    'PHI' : 'AWAY_PHI', 
    'PHX' : 'AWAY_PHX', 
    'POR' : 'AWAY_POR', 
    'SAC' : 'AWAY_SAC', 
    'SAS' : 'AWAY_SAS', 
    'TOR' : 'AWAY_TOR', 
    'UTA' : 'AWAY_UTA', 
    'WAS' : 'AWAY_WAS',  
}
AWAY_TEAM = pd.get_dummies(df.AWAY_TEAM)
AWAY_TEAM = AWAY_TEAM.rename(columns=AWAY_TEAM_DICT)

HOME_TEAM = pd.get_dummies(df.HOME_TEAM)

model_df = df.copy()
model_df = model_df.drop(columns=['GAME_DATE', 'GAME_ID', 'MATCHUP', 'HOME_TEAM', 'AWAY_TEAM'])

df_model = pd.concat((
    HOME_TEAM, 
    AWAY_TEAM,
    model_df
), axis=1)

In [6]:
df_model.head()

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DEN,DET,GSW,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NOP,NYK,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS,AWAY_ATL,AWAY_BKN,AWAY_BOS,AWAY_CHA,AWAY_CHI,AWAY_CLE,AWAY_DAL,AWAY_DEN,AWAY_DET,AWAY_GSW,AWAY_HOU,AWAY_IND,AWAY_LAC,AWAY_LAL,AWAY_MEM,AWAY_MIA,AWAY_MIL,AWAY_MIN,AWAY_NOP,AWAY_NYK,AWAY_OKC,AWAY_ORL,AWAY_PHI,AWAY_PHX,AWAY_POR,AWAY_SAC,AWAY_SAS,AWAY_TOR,AWAY_UTA,AWAY_WAS,HOME_PREV_SEAS_WIN_PERC,HOME_PREV_SEAS_AVG_PTS_PER_GAME,HOME_PREV_SEAS_FG_PERC,HOME_PREV_SEAS_THREE_PERC,HOME_PREV_SEAS_FREE_THROW_PERC,HOME_PREV_SEAS_PLUS_MINUS,AWAY_PREV_SEAS_WIN_PERC,AWAY_PREV_SEAS_AVG_PTS_PER_GAME,AWAY_PREV_SEAS_FG_PERC,AWAY_PREV_SEAS_THREE_PERC,AWAY_PREV_SEAS_FREE_THROW_PERC,AWAY_PREV_SEAS_PLUS_MINUS,HOME_WIN
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.605,94.7,43.6,34.7,74.6,4.0,0.244,94.1,44.8,32.9,75.5,-7.0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.549,102.2,45.8,35.5,69.2,1.2,0.683,101.1,47.8,35.8,71.1,6.5,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.805,102.9,49.6,39.6,75.4,7.9,0.549,93.2,43.7,35.3,77.3,0.3,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.305,95.2,44.3,33.0,74.4,-6.5,0.402,97.5,44.8,35.3,77.6,-3.2,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.341,100.2,44.7,36.3,76.9,-4.9,0.695,106.1,47.8,34.3,70.1,5.1,1


In [7]:
X = df_model.drop(columns=['HOME_WIN'])
y = df_model['HOME_WIN']

In [8]:
from sklearn import preprocessing
mms = preprocessing.MinMaxScaler()
temp_df = X.copy()
for i in temp_df.columns[60:]:
  #print(i)
  X[i] = mms.fit_transform(temp_df[i].values.reshape(-1,1))

In [9]:
X

,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DEN,DET,GSW,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NOP,NYK,OKC,ORL,PHI,PHX,POR,SAC,SAS,TOR,UTA,WAS,AWAY_ATL,AWAY_BKN,AWAY_BOS,AWAY_CHA,AWAY_CHI,AWAY_CLE,AWAY_DAL,AWAY_DEN,AWAY_DET,AWAY_GSW,AWAY_HOU,AWAY_IND,AWAY_LAC,AWAY_LAL,AWAY_MEM,AWAY_MIA,AWAY_MIL,AWAY_MIN,AWAY_NOP,AWAY_NYK,AWAY_OKC,AWAY_ORL,AWAY_PHI,AWAY_PHX,AWAY_POR,AWAY_SAC,AWAY_SAS,AWAY_TOR,AWAY_UTA,AWAY_WAS,HOME_PREV_SEAS_WIN_PERC,HOME_PREV_SEAS_AVG_PTS_PER_GAME,HOME_PREV_SEAS_FG_PERC,HOME_PREV_SEAS_THREE_PERC,HOME_PREV_SEAS_FREE_THROW_PERC,HOME_PREV_SEAS_PLUS_MINUS,AWAY_PREV_SEAS_WIN_PERC,AWAY_PREV_SEAS_AVG_PTS_PER_GAME,AWAY_PREV_SEAS_FG_PERC,AWAY_PREV_SEAS_THREE_PERC,AWAY_PREV_SEAS_FREE_THROW_PERC,AWAY_PREV_SEAS_PLUS_MINUS
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.643494,0.116279,0.154930,0.428571,0.397059,0.717391,0.000000,0.069767,0.323944,0.244898,0.463235,0.119565
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.543672,0.697674,0.464789,0.510204,0.000000,0.565217,0.782531,0.612403,0.746479,0.540816,0.139706,0.853261
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.751938,1.000000,0.928571,0.455882,0.929348,0.543672,0.000000,0.169014,0.489796,0.595588,0.516304
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.108734,0.155039,0.253521,0.255102,0.382353,0.146739,0.281640,0.333333,0.323944,0.489796,0.617647,0.326087
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.172906,0.542636,0.309859,0.591837,0.566176,0.233696,0.803922,1.000000,0.746479,0.387755,0.066176,0.777174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.869875,0.968992,0.788732,0.734694,1.000000,1.000000,0.196078,0.131783,0.338028,0.520408,0.051471,0.282609
1226,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.172906,0.542636,0.309859,0.591837,0.566176,0.233696,0.108734,0.155039,0.253521,0.255102,0.382353,0.146739
1227,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.390374,0.441860,0.140845,0.561224,0.323529,0.418478,0.522282,0.372093,0.549296,0.673469,0.169118,0.521739
1228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.281640,0.333333,0.323944,0.489796,0.617647,0.326087,0.782531,0.612403,0.746479,0.540816,0.139706,0.853261


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [12]:
# simple logistic regression model similar to the baseline classifier
clf_13 = LogisticRegression(random_state=101, max_iter=1000).fit(X_train, y_train)

In [13]:
clf_13.score(X_test, y_test)

0.6829268292682927

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
parameters = {
    'n_estimators' : [150, 175, 187, 200, 213, 225, 250],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [7, 8, 9, 10, 11],
    'max_features' : ['auto']
    }

In [16]:
rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, parameters, cv=5, verbose=3)

In [17]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 70 candidates, totalling 350 fits
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=150 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=150, score=0.624, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=150 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=150, score=0.634, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=150 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=150, score=0.634, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=150 
[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=150, score=0.645, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=150 
[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=150, score=0.686, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=175 
[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=175, score=0.595, total=   0.3s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=175 
[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=175, score=0.634, total=   0.4s
[CV] criterion=gini, max_depth=7, max_features=auto, n_estimators=175 
[CV]  criterion=gini, max_depth=7, max_features=auto, n_estimators=175, score=0.616, total=   0.4s
[CV] criterion=gini, max_depth=7, max_features=auto

[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [18]:
results = clf.cv_results_

In [19]:
clf.best_estimator_


# 0.65 test
# RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='entropy', max_depth=9, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=175,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=213,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
clf.score(X_train, y_train)

0.8524970963995354

In [21]:
clf.score(X_test, y_test)

0.6449864498644986

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

parameters = {
    'learning_rate' : [0.01, 0.3],
    'n_estimators' : [325, 750],
    'max_depth' : [4, 5, 6],
    'max_features' : ['auto']
    }

In [23]:
gbc = GradientBoostingClassifier()
gbc_clf = GridSearchCV(gbc, parameters, cv=5, verbose=3)

In [24]:
gbc_clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325, score=0.578, total=   0.9s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325, score=0.634, total=   0.9s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325, score=0.692, total=   0.9s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325 
[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325, score=0.640, total=   0.9s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325 
[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=325, score=0.674, total=   0.9s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750 
[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750, score=0.572, total=   2.0s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750 
[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750, score=0.663, total=   2.0s
[CV] learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750 
[CV]  learning_rate=0.01, max_depth=4, max_features=auto, n_estimators=750, score=0.686, total=   2.0s
[CV] le

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [25]:
gbc_clf.score(X_train, y_train)

0.8339140534262486

In [26]:
gbc_clf.score(X_test, y_test)

0.6449864498644986

In [27]:
results = gbc_clf.cv_results_
gbc_clf.best_estimator_

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=5,
                           max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=325,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)